In [6]:
import pandas as pd
from imdb import IMDb
from tqdm import tqdm

Надо достать:
- для фильмов:
    * Название,
    * Дату выхода,
    * Слоган,
    * Бюджет,
    * Выручку,
    * Продолжительность,
    * Рейтинг,
    * Описание,
    * Жанры,
    * Страны,
    * Персонал: id человека, его роль при создании фильма, исполняемый персонаж при наличии.
- для сериалов:
    * Название,
    * Дату выхода,
    * Слоган,
    * Бюджет,
    * Рейтинг,
    * Описание,
    * Жанры,
    * Страны,
    * Персонал.
    * Для каждого эпизода:
        * Номер сезона,
        * Номер эпизода в сезоне,
        * Название,
        * Дата выхода,
        * Продолжительность,
        * Рейтинг.
- Для персонала:
    * Имя,
    * Дату рождения,
    * Дату смерти (при наличии),
    * Место рождения,
    * Описание.

In [7]:
ia = IMDb()

In [8]:
file_path = 'data.csv'
df = pd.read_csv(file_path)

In [9]:
movie_results = []
personnel_data = []

In [21]:
for index, row in tqdm(df.iterrows()):
    movie_id = row['Const'][2:]  # Убираем префикс 'tt'
    movie = ia.get_movie(movie_id)
    ia.update(movie, info=['main', 'plot', 'taglines'])

    # Сбор данных о фильме
    movie_data = {
        'MovieID': movie_id,
        'Title': movie.get('title'),
        'Release Date': movie.get('year'),
        'Tagline': movie.get('taglines')[0] if movie.get('taglines') else None,
        'Budget': movie.get('box office', {}).get('Budget'),
        'Revenue': movie.get('box office', {}).get('Cumulative Worldwide Gross'),
        'Duration': movie.get('runtimes')[0] if movie.get('runtimes') else None,
        'Rating': movie.get('rating'),
        'Description': movie.get('plot')[0].split('::')[0] if movie.get('plot') else None,
        'Genres': movie.get('genres', []),
        'Countries': movie.get('countries', []),
    }
    movie_results.append(movie_data)

    break
    # Сбор данных о персонале
    for role, role_key in [('Director', 'directors'), ('Writer', 'writers'), ('Actor', 'cast')]:
        for person in movie.get(role_key, []):
            personnel_data.append({
                'PersonID': person.personID,
                'Name': str(person),
                'Role': role,
                'MovieID': movie_id,
                'Character': ', '.join(person.currentRole) if getattr(person, 'currentRole', None) else None
            })
    break

0it [00:07, ?it/s]


In [22]:
movie_results

[{'MovieID': '0816692',
  'Title': 'Interstellar',
  'Release Date': 2014,
  'Tagline': 'Mankind was born on Earth. It was never meant to die here.',
  'Budget': '$165,000,000 (estimated)',
  'Revenue': '$675,020,017',
  'Duration': '169',
  'Rating': 8.7,
  'Description': 'When Earth becomes uninhabitable in the future, a farmer and ex-NASA pilot, Joseph Cooper, is tasked to pilot a spacecraft, along with a team of researchers, to find a new planet for humans.',
  'Genres': 'Adventure, Drama, Sci-Fi',
  'Countries': 'United States, United Kingdom, Canada'},
 {'MovieID': '0816692',
  'Title': 'Interstellar',
  'Release Date': 2014,
  'Tagline': 'Mankind was born on Earth. It was never meant to die here.',
  'Budget': '$165,000,000 (estimated)',
  'Revenue': '$675,020,017',
  'Duration': '169',
  'Rating': 8.7,
  'Description': 'When Earth becomes uninhabitable in the future, a farmer and ex-NASA pilot, Joseph Cooper, is tasked to pilot a spacecraft, along with a team of researchers, to

In [15]:
movie.infoset2keys

{'main': ['original title',
  'localized title',
  'cast',
  'genres',
  'runtimes',
  'countries',
  'country codes',
  'language codes',
  'color info',
  'aspect ratio',
  'sound mix',
  'box office',
  'certificates',
  'original air date',
  'rating',
  'votes',
  'cover url',
  'imdbID',
  'videos',
  'plot outline',
  'languages',
  'title',
  'year',
  'kind',
  'director',
  'writer',
  'producer',
  'composer',
  'cinematographer',
  'editor',
  'editorial department',
  'casting director',
  'production design',
  'art direction',
  'set decoration',
  'costume designer',
  'make up',
  'production manager',
  'assistant director',
  'art department',
  'sound crew',
  'special effects',
  'visual effects',
  'stunt performer',
  'camera and electrical department',
  'casting department',
  'costume department',
  'location management',
  'music department',
  'script department',
  'transportation department',
  'miscellaneous crew',
  'thanks',
  'akas',
  'top 250 rank',


In [ ]:
movies_df = pd.DataFrame(movie_results)
movies_df.to_csv('movies_data.csv', index=False)

personnel_df = pd.DataFrame(personnel_data)
personnel_df.to_csv('personnel_data.csv', index=False)

In [ ]:
pd.read_csv('movies_data.csv').head()

In [ ]:
pd.read_csv('personnel_data.csv').head()